In [1]:
import pandas as pd
import similaripy as sim
from scipy import *
from scipy.sparse import *
from tqdm import tqdm
import numpy as np

import re
import string as string_lib
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer

In [19]:
def ngrams(string, n=3):
    string = string.encode("ascii", errors="ignore").decode() #remove non ascii chars
    string = string.lower() #make lower case
    chars_to_remove = [")","(","|","[","]","{","}","'"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(rx, '', string) #remove the list of chars defined above
    string = re.sub(' +',' ',string).strip() # get rid of multiple spaces and replace with a single space
    string = ' '+ string +' ' # pad names for ngrams...
    string = re.sub(r'[,-/]',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

In [3]:
# first load the data
df_train = pd.read_csv("../dataset/original/train.csv", escapechar="\\")
df_test = pd.read_csv("../dataset/original/test.csv", escapechar="\\")
# ALWAYS sort the data by record_id
df_train = df_train.sort_values(by=['record_id']).reset_index(drop=True)
df_test = df_test.sort_values(by=['record_id']).reset_index(drop=True)

In [4]:
df_train.head()

,record_id,name,type,address,phone,email,modification,linked_id
0,10000001,"TIANSHENG INDUSTRY AND TRADING CO., LTD.",entity,NaN,3.391448e+11,inquires@TIANSHENG.INDUSTRY.AND.TRADING.CO.LTD.de,original,10000001
1,10000002,"NINGBO SUNRISE ENTERPRISES UNITED CO., LTD.",entity,NaN,1.813704e+10,consulting@zoho.cz,original,10000002
2,10000002-M0,"NINGBO SUNRISE ENTERPRISES UNITED CO., LTD.",entity,NaN,1.813704e+10,NaN,missing,10000002
3,10000002-M1,"NINGBO SUNRISE ENTERPRISES UNITED CO., LTD.",entity,NaN,NaN,NaN,missing,10000002
4,10000004,"SKY-BLUE GIFTS & TOYS CO., LTD.",entity,NaN,NaN,NaN,original,10000004


In [6]:
df_test = convert_phones(df_test)
df_train = convert_phones(df_train)

In [7]:
# mi serve una colonna con tutti i telefoni su cui fare tfidf
all_phones = list(df_train.phone) + list(df_test.phone)
# daje con tfidf
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(all_phones)

In [8]:
# split
tf_idf_train = tf_idf_matrix[:691440,:] # 691440 è la lunghezza del train
tf_idf_test = tf_idf_matrix[691440:,:]
jac = sim.jaccard(tf_idf_test, tf_idf_train.T, k=300)
save_npz('jaccard_tfidf_phone.npz', jac.tocsr())

Computing:   8%|████▏                                                   | 20254/266955 [01:12<14:44, 278.92it/s]

KeyboardInterrupt: 